In [119]:
import functions

In [120]:
from platform import python_version

print(python_version())

3.7.3


In [121]:
import numpy as np
import csv
import math

def get_family_p(country):
    with open('Houseshold.csv',encoding='latin-1') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        flag=False
        for row in csv_reader:
            if flag==False:
                if row[0]==country and len(row[0])==len(country):
                    family_vector=np.array(row)
                    flag=True
            else:
                if row[0]==country and len(row[0])==len(country):
                    family_vector=np.array(row)
                else:
                    break
    family_vector[family_vector=='..']='0.0'
    average_household_size=float(family_vector[4])
    one_person_p=float(family_vector[30])/100
    couple_only_p=float(family_vector[31])/100
    couple_with_children_p=float(family_vector[32])/100
    single_parent_with_children_p=float(family_vector[33])/100
    average_household_size-1*one_person_p-2*couple_only_p
    Extended_family_p=float(family_vector[36])/100
    Non_relative_p=float(family_vector[37])/100
    return np.array([one_person_p,couple_only_p,couple_with_children_p,single_parent_with_children_p])

def get_age_distribution(country):
    age_distribution=[]
    with open('World_Age_2019.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if row[0]==country:
                for i in range(101):
                    age_distribution.append(float(row[i+1]))
                break
    return np.array(age_distribution)

def get_mean_child_number(country):
    with open('TotalFertility.csv',encoding='latin-1') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if row[0]==country:
                child_number=float(row[1])
                break
    return child_number

def get_mother_birth_age_distribution(country):
    mother_birth_age_distribution=[]
    with open('AgeSpecificFertility.csv',encoding='latin-1') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if row[0]==country:
                #15-19	20-24	25-29	30-34	35-39	40-44	45-49
                for i in range(7):
                    mother_birth_age_distribution.append(float(row[i+1]))
                break
    return np.array(mother_birth_age_distribution)
"""
def sample_households_un(n,country):
    max_household_size = 8
    households = np.zeros((n, max_household_size), dtype=np.int)
    households[:] = -1
    age = np.zeros(n, dtype=np.int)
    n_ages = 101
    
    age_distribution = get_age_distribution(country)
    age_distribution = np.array(age_distribution)
    age_distribution = age_distribution/age_distribution.sum()
        
    household_probs = get_family_p(country)
    household_probs /= household_probs.sum()
    num_generated = 0
    #All from UN data
    mean_child_number=get_mean_child_number(country)
    mother_birth_age_distribution=get_mother_birth_age_distribution(country)
    child_to_adult=25
    while num_generated < n:
        if n - num_generated < 5:
            i = 0
        else:
            i = np.random.choice(household_probs.shape[0], p=household_probs)
        #single person household
        #sample from age distribution
        if i == 0:
        #The couple will become couple with children once mother give birth and back to couple when children leave house
            renormalized = mother_birth_age_distribution/mother_birth_age_distribution.sum() 
            mother_age_at_birth = (np.random.choice(7, p=renormalized) + 3)*5+np.random.randint(5)
            if mother_age_at_birth>child_to_adult:
                renormalized = np.append(age_distribution[child_to_adult:mother_age_at_birth],age_distribution[(mother_age_at_birth+child_to_adult):])
                age_vec=np.append(range(child_to_adult,mother_age_at_birth) ,range((mother_age_at_birth+child_to_adult),n_ages))
            else:
                renormalized =age_distribution[mother_age_at_birth+child_to_adult:]
                age_vec=range((mother_age_at_birth+child_to_adult),n_ages)
            renormalized = renormalized/renormalized.sum()
            age[num_generated]=age_vec[np.random.choice(len(age_vec), p=renormalized)]
#            renormalized = age_distribution[child_to_adult:]
#            renormalized = renormalized/renormalized.sum()
#            age[num_generated] = np.random.choice(n_ages-child_to_adult, p=renormalized)+child_to_adult
            generated_this_step = 1
        #couple, sample from age distribution conditioned on age >= 22
        elif i == 1:
            renormalized = mother_birth_age_distribution/mother_birth_age_distribution.sum() 
            mother_age_at_birth = (np.random.choice(7, p=renormalized) + 3)*5+np.random.randint(5)
            if mother_age_at_birth>child_to_adult:
                renormalized = np.append(age_distribution[child_to_adult:mother_age_at_birth],age_distribution[(mother_age_at_birth+child_to_adult):])
                age_vec=np.append(range(child_to_adult,mother_age_at_birth) ,range((mother_age_at_birth+child_to_adult),n_ages))
            else:
                renormalized =age_distribution[mother_age_at_birth+child_to_adult:]
                age_vec=range((mother_age_at_birth+child_to_adult),n_ages)
            renormalized = renormalized/renormalized.sum()
            age[num_generated]=age_vec[np.random.choice(len(age_vec), p=renormalized)]
            age[num_generated+1]=age_vec[np.random.choice(len(age_vec), p=renormalized)]
#            renormalized = age_distribution[(child_to_adult):]
#            renormalized = renormalized/renormalized.sum()
#            age[num_generated] = np.random.choice(n_ages-(child_to_adult), p=renormalized) + (child_to_adult)
#            age[num_generated+1] = np.random.choice(n_ages-(child_to_adult), p=renormalized) + (child_to_adult)
            generated_this_step = 2
        #couple with children
        elif i == 2:           
            child_number=min(2,np.random.poisson(mean_child_number-1))
            child_number+=1
            child_ages=[]
            for j in range(child_number):
                renormalized = age_distribution[:child_to_adult]
                renormalized = renormalized/renormalized.sum()
                child_age = np.random.choice(child_to_adult, p=renormalized)
                age[num_generated+j] = child_age
                child_ages.append(child_age)
            renormalized = mother_birth_age_distribution/mother_birth_age_distribution.sum() 
            mother_age_at_birth = (np.random.choice(7, p=renormalized) + 3)*5+np.random.randint(5)
            mother_current_age = mother_age_at_birth + max(child_ages)
            age[num_generated+child_number] = mother_current_age
            age[num_generated+child_number + 1] = mother_current_age
            generated_this_step = 2+child_number
        #single parent with children_p
        elif i == 3:
            child_number=min(2,np.random.poisson(mean_child_number-1))
            child_number+=1
            child_ages=[]
            for j in range(child_number):
                renormalized = age_distribution[:child_to_adult]
                renormalized = renormalized/renormalized.sum()
                child_age = np.random.choice(child_to_adult, p=renormalized)
                age[num_generated+j] = child_age
                child_ages.append(child_age)
            renormalized = mother_birth_age_distribution/mother_birth_age_distribution.sum()
            mother_age_at_birth = (np.random.choice(7, p=renormalized) + 3)*5+np.random.randint(5)
            mother_current_age = mother_age_at_birth + max(child_ages)
            age[num_generated+child_number] = mother_current_age
            generated_this_step = 1+child_number
        #update list of household contacts
        for i in range(num_generated, num_generated+generated_this_step):
            curr_pos = 0
            for j in range(num_generated, num_generated+generated_this_step):
                if i != j:
                    households[i, curr_pos] = j
                    curr_pos += 1
        num_generated += generated_this_step
    return households, age
    
    

def sample_households_china(n):
    max_household_size = 4

    households = np.zeros((n, max_household_size), dtype=np.int)
    households[:] = -1
    age = np.zeros(n, dtype=np.int)
    n_ages = 101
    #estimates for china from 2020
    #from https://population.un.org/wpp/Download/Standard/Interpolated/
    age_distribution = [16113.281,16543.361,16875.302,17118.429,17282.064,17375.527,17408.145,17389.238,17328.13,17234.143,17117.175,16987.122,16850.435,16715.289,16592.73,16484.473,16388.786,16370.261,16460.9,16637.439,16866.861,17182.465,17477.132,17702.896,17928.813,18144.994,18201.129,18841.832,20387.657,22413.391,24308.028,26355.485,27269.657,26400.295,24405.505,22597.72,20719.355,19296.916,18726.536,18750.928,18640.938,18451.511,18716.505,19599.644,20865.548,22101.75,23374.699,24376.638,24907.095,25077.435,25250.357,25414.362,25172.526,24383.003,23225.134,22043.117,20795.729,19608.86,18589.082,17703.703,16743.545,15666.543,14988.213,14917.427,15198.411,15425.124,15749.105,15550.741,14503.063,12921.733,11444.972,9939.85,8651.521,7764.623,7148.723,6478.704,5807.535,5222.027,4729.055,4307.295,3931.038,3608.42,3272.336,2887.659,2481.964,2118.152,1783.88,1480.587,1215.358,983.8,739.561,551.765,453.96,342.463,217.275,145.809,122.178,96.793,69.654,40.759,74.692]
    age_distribution = np.array(age_distribution)
    age_distribution = age_distribution/age_distribution.sum()
    
    #single person, couple only, parents and unmarried children, 3-generation
    #from https://link.springer.com/article/10.1186/s40711-015-0011-0/tables/2
    #(2010 census, urban populations)
    household_probs = np.array([0.1703, .2117, 0.3557, 0.1126])
    household_probs /= household_probs.sum()
    num_generated = 0
    while num_generated < n:
        if n - num_generated < 5:
            i = 0
        else:
            i = np.random.choice(household_probs.shape[0], p=household_probs)
        #single person household
        #sample from age distribution
        if i == 0:
            age[num_generated] = np.random.choice(n_ages, p=age_distribution)
            generated_this_step = 1
        #couple, sample from age distribution conditioned on age >= 22
        elif i == 1:
            renormalized = age_distribution[22:]
            renormalized = renormalized/renormalized.sum()
            age[num_generated] = np.random.choice(n_ages-22, p=renormalized) + 22
            age[num_generated+1] = np.random.choice(n_ages-22, p=renormalized) + 22
            generated_this_step = 2
        #some information about mother's age at birth of first child
        #https://link.springer.com/article/10.1007/s42379-019-00022-9
        elif i == 2:
            renormalized = age_distribution[:22]
            renormalized = renormalized/renormalized.sum()
            child_age = np.random.choice(22, p=renormalized)
            age[num_generated] = child_age
            #super rough approximation, women have child at a uniformly random age between 23 and 33
            renormalized = age_distribution[23:34]
            renormalized = renormalized/renormalized.sum()
            mother_age_at_birth = np.random.choice(11, p=renormalized) + 23
            mother_current_age = mother_age_at_birth + child_age
            age[num_generated + 1] = mother_current_age
            age[num_generated + 2] = mother_current_age
            generated_this_step = 3
        elif i == 3:
            #start by generating parents/unmarried child
            renormalized = age_distribution[:22]
            renormalized = renormalized/renormalized.sum()
            child_age = np.random.choice(22, p=renormalized)
            age[num_generated] = child_age
            #super rough approximation, women have child at a uniformly random age between 23 and 33
            renormalized = age_distribution[23:34]
            renormalized = renormalized/renormalized.sum()
            mother_age_at_birth = np.random.choice(11, p=renormalized) + 23
            mother_current_age = mother_age_at_birth + child_age
            age[num_generated + 1] = mother_current_age
            age[num_generated + 2] = mother_current_age
            #add grandparents
            renormalized = age_distribution[23:34]
            renormalized = renormalized/renormalized.sum()
            grandmother_age_at_birth = np.random.choice(11, p=renormalized) + 23
            grandmother_current_age = grandmother_age_at_birth + mother_current_age
            age[num_generated + 3] = grandmother_current_age
            age[num_generated + 4] = grandmother_current_age
            generated_this_step = 5
        
        #update list of household contacts
        for i in range(num_generated, num_generated+generated_this_step):
            curr_pos = 0
            for j in range(num_generated, num_generated+generated_this_step):
                if i != j:
                    households[i, curr_pos] = j
                    curr_pos += 1
        num_generated += generated_this_step
    return households, age

def sample_households_italy(n):
    max_household_size = 6
    
    households = np.zeros((n, max_household_size), dtype=np.int)
    households[:] = -1
    age = np.zeros(n, dtype=np.int)    
    n_ages = 101
        
    age_distribution = get_age_distribution("Italy")
    age_distribution = np.array(age_distribution)
    age_distribution = age_distribution/age_distribution.sum()
        
    # List of household types: single household, couple without children, single parent +1/2/3 children, couple +1/2/3 children,
    # family without a nucleus, nucleus with other persons, households with two or more nuclei (a and b)
    household_probs = np.array([0.308179, 0.191000, 0.0694283, 0.0273065, 0.00450268, 0.152655, 0.132429, 0.0340969, 
                       0.043821, 0.033, 0.0150])
    household_probs /= household_probs.sum()
    
    num_generated = 0
    
    # from fertility data
    mother_birth_age_distribution=get_mother_birth_age_distribution("Italy")    
    renormalized_mother = mother_birth_age_distribution/mother_birth_age_distribution.sum()
    renormalized_adult = age_distribution[18:]
    renormalized_adult = renormalized_adult/renormalized_adult.sum()
    # 18 considered as majority age, maybe should consider that children may still live with parents until 30 or so
    renormalized_child = age_distribution[:30]
    renormalized_child = renormalized_child/renormalized_child.sum()
    
    renormalized_adult_older = age_distribution[30:]
    renormalized_adult_older /= renormalized_adult_older.sum()
    # 60 considered as retirement threshold, maybe should be larger, but reasonable for first pass
    renormalized_grandparent = age_distribution[60:]
    renormalized_grandparent = renormalized_grandparent/renormalized_grandparent.sum()
    
    while num_generated < n:
        if n - num_generated < (max_household_size+1):
            i = 0
        else:
            i = np.random.choice(household_probs.shape[0], p=household_probs)
        #single person household
        #sample from age distribution
        if i == 0:
            age[num_generated] = np.random.choice(n_ages-30, p=renormalized_adult_older) + 30
            generated_this_step = 1
        # couple, sample from age distribution conditioned on age >= 18
        elif i == 1:  
            age_adult = np.random.choice(n_ages-30, p=renormalized_adult_older) + 30
            age[num_generated] = age_adult
            age[num_generated+1] = min(n_ages-1,age_adult+3) # man three years older on average
            generated_this_step = 2
        # single parent, 1 child
        elif i == 2:            
            child_age = np.random.choice(30, p=renormalized_child)
            age[num_generated] = child_age
            #super rough approximation
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + child_age)
            age[num_generated + 1] = mother_current_age
            generated_this_step = 2
        # single parent, 2 children
        elif i == 3:
            for j in range(2):                
                child_age = np.random.choice(30, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+2)]))
            age[num_generated + 2] = mother_current_age
            generated_this_step = 3
        # single parent, 3 children
        elif i == 4:
            for j in range(3):                
                child_age = np.random.choice(30, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+3)]))
            age[num_generated + 3] = mother_current_age
            generated_this_step = 4
            
        # couple, 1 child
        elif i == 5: 
            child_age = np.random.choice(30, p=renormalized_child)
            age[num_generated] = child_age
            #super rough approximation
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + child_age)
            age[num_generated + 1] = mother_current_age
            age[num_generated + 2] = min(n_ages-1,mother_current_age+3)
            generated_this_step = 3
        
        # couple, 2 children
        elif i == 6:
            for j in range(2):                
                child_age = np.random.choice(30, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+2)]))
            age[num_generated + 2] = mother_current_age
            age[num_generated + 3] = min(n_ages-1,mother_current_age+3)
            generated_this_step = 4            
        
        # couple, 3 children
        elif i == 7:
            for j in range(3):                
                child_age = np.random.choice(30, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+3)]))
            age[num_generated + 3] = mother_current_age
            age[num_generated + 4] = min(n_ages-1,mother_current_age+3)
            generated_this_step = 5
        
        # family without nucleus
        elif i == 8:
            age[num_generated] = np.random.choice(n_ages-30, p=renormalized_adult_older) + 30
            age[num_generated+1] = np.random.choice(n_ages-30, p=renormalized_adult_older) + 30
            generated_this_step = 2         
                
        # nucleus with other persons (couple, 2 children, adult >= 60)
        elif i == 9:
            for j in range(2):                
                child_age = np.random.choice(30, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+2)]))
            age[num_generated + 2] = mother_current_age
            age[num_generated + 3] = min(n_ages-1,mother_current_age+3)
            age[num_generated + 4] = np.random.choice(n_ages-60, p=renormalized_grandparent) + 60
            generated_this_step = 5
            
        # households with 2 or more nuclei
        # a - couple with same age for mother/father sampled from > 18 + 2 children <= 18 + 2 grand-parents
        # b - couple with same age for mother/father sampled from > 18 + 2 children <= 18 + 2 children from other marriage <= 18
        # scenario b removed for now 
        
        elif i == 10:
            for j in range(2):                
                child_age = np.random.choice(30, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+2)]))
            age[num_generated + 2] = mother_current_age
            age[num_generated + 3] = min(n_ages-1,mother_current_age+3)
            #grandparent_age =  np.random.choice(n_ages-60, p=renormalized_grandparent) + 60
            grandmother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            grandmother_current_age = min(n_ages-1,grandmother_age_at_birth + mother_current_age)
            #age[num_generated + 4] = grandparent_age
            #age[num_generated + 5] = grandparent_age+3   
            age[num_generated + 4] = grandmother_current_age
            age[num_generated + 5] = min(n_ages-1,grandmother_current_age+3)   
            generated_this_step = 6
            
        #elif i == 11:
            #for j in range(4):                
                #child_age = np.random.choice(30, p=renormalized_child)
                #age[num_generated+j] = child_age
            #mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            #mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+4)]))
            #age[num_generated + 4] = mother_current_age
            #age[num_generated + 5] = min(n_ages-1,mother_current_age+3)          
            #generated_this_step = 6
            
        #update list of household contacts
        for i in range(num_generated, num_generated+generated_this_step):
            curr_pos = 0
            for j in range(num_generated, num_generated+generated_this_step):
                if i != j:
                    households[i, curr_pos] = j
                    curr_pos += 1
        num_generated += generated_this_step
        
    return households, age

def sample_households_lombardy(n):
    max_household_size = 6
    
    households = np.zeros((n, max_household_size), dtype=np.int)
    households[:] = -1
    age = np.zeros(n, dtype=np.int)    
    n_ages = 101
        
    age_distribution = [0.00786667, 0.00786667, 0.00786667, 0.0087, 0.0087,
       0.0087    , 0.00963333, 0.00963333, 0.00963333, 0.00963333,
       0.00963333, 0.00963333, 0.0095    , 0.0095    , 0.0095    ,
       0.0095    , 0.0095    , 0.0095    , 0.00945714, 0.00945714,
       0.00945714, 0.00945714, 0.00945714, 0.00945714, 0.00945714,
       0.01058   , 0.01058   , 0.01058   , 0.01058   , 0.01058   ,
       0.01058   , 0.01058   , 0.01058   , 0.01058   , 0.01058   ,
       0.01381   , 0.01381   , 0.01381   , 0.01381   , 0.01381   ,
       0.01381   , 0.01381   , 0.01381   , 0.01381   , 0.01381   ,
       0.01664   , 0.01664   , 0.01664   , 0.01664   , 0.01664   ,
       0.01664   , 0.01664   , 0.01664   , 0.01664   , 0.01664   ,
       0.0133    , 0.0133    , 0.0133    , 0.0133    , 0.0133    ,
       0.0133    , 0.0133    , 0.0133    , 0.0133    , 0.0133    ,
       0.01091   , 0.01091   , 0.01091   , 0.01091   , 0.01091   ,
       0.01091   , 0.01091   , 0.01091   , 0.01091   , 0.01091   ,
       0.00894924, 0.00859131, 0.00823338, 0.00787545, 0.00751752,
       0.00715959, 0.00680166, 0.00644373, 0.0060858 , 0.00572787,
       0.00536994, 0.00501201, 0.00465408, 0.00429615, 0.00393822,
       0.0035803 , 0.00322237, 0.00286444, 0.00250651, 0.00214858,
       0.00179065, 0.00143272, 0.00107479, 0.00071686, 0.00035893,
       0.00054791]
    age_distribution = np.array(age_distribution)
    age_distribution = age_distribution/age_distribution.sum()
        
    # List of household types: single household, couple without children, single parent +1/2/3 children, couple +1/2/3 children,
    # family without a nucleus, nucleus with other persons, households with two or more nuclei (a and b)
    household_probs = np.array([0.308179, 0.191000, 0.0694283, 0.0273065, 0.00450268, 0.152655, 0.132429, 0.0340969, 
                       0.043821, 0.033, 0.0150])
    household_probs /= household_probs.sum()
    
    num_generated = 0
    
    # from fertility data
    mother_birth_age_distribution=get_mother_birth_age_distribution("Italy")    
    renormalized_mother = mother_birth_age_distribution/mother_birth_age_distribution.sum()
    renormalized_adult = age_distribution[18:]
    renormalized_adult = renormalized_adult/renormalized_adult.sum()
    # 18 considered as majority age, maybe should consider that children may still live with parents until 30 or so
    renormalized_child = age_distribution[:30]
    renormalized_child = renormalized_child/renormalized_child.sum()
    
    renormalized_adult_older = age_distribution[30:]
    renormalized_adult_older /= renormalized_adult_older.sum()
    # 60 considered as retirement threshold, maybe should be larger, but reasonable for first pass
    renormalized_grandparent = age_distribution[60:]
    renormalized_grandparent = renormalized_grandparent/renormalized_grandparent.sum()
    
    while num_generated < n:
        if n - num_generated < (max_household_size+1):
            i = 0
        else:
            i = np.random.choice(household_probs.shape[0], p=household_probs)
        #single person household
        #sample from age distribution
        if i == 0:
            age[num_generated] = np.random.choice(n_ages-30, p=renormalized_adult_older) + 30
            generated_this_step = 1
        # couple, sample from age distribution conditioned on age >= 18
        elif i == 1:  
            age_adult = np.random.choice(n_ages-30, p=renormalized_adult_older) + 30
            age[num_generated] = age_adult
            age[num_generated+1] = min(n_ages-1,age_adult+3) # man three years older on average
            generated_this_step = 2
        # single parent, 1 child
        elif i == 2:            
            child_age = np.random.choice(30, p=renormalized_child)
            age[num_generated] = child_age
            #super rough approximation
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + child_age)
            age[num_generated + 1] = mother_current_age
            generated_this_step = 2
        # single parent, 2 children
        elif i == 3:
            for j in range(2):                
                child_age = np.random.choice(30, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+2)]))
            age[num_generated + 2] = mother_current_age
            generated_this_step = 3
        # single parent, 3 children
        elif i == 4:
            for j in range(3):                
                child_age = np.random.choice(30, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+3)]))
            age[num_generated + 3] = mother_current_age
            generated_this_step = 4
            
        # couple, 1 child
        elif i == 5: 
            child_age = np.random.choice(30, p=renormalized_child)
            age[num_generated] = child_age
            #super rough approximation
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + child_age)
            age[num_generated + 1] = mother_current_age
            age[num_generated + 2] = min(n_ages-1,mother_current_age+3)
            generated_this_step = 3
        
        # couple, 2 children
        elif i == 6:
            for j in range(2):                
                child_age = np.random.choice(30, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+2)]))
            age[num_generated + 2] = mother_current_age
            age[num_generated + 3] = min(n_ages-1,mother_current_age+3)
            generated_this_step = 4            
        
        # couple, 3 children
        elif i == 7:
            for j in range(3):                
                child_age = np.random.choice(30, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+3)]))
            age[num_generated + 3] = mother_current_age
            age[num_generated + 4] = min(n_ages-1,mother_current_age+3)
            generated_this_step = 5
        
        # family without nucleus
        elif i == 8:
            age[num_generated] = np.random.choice(n_ages-30, p=renormalized_adult_older) + 30
            age[num_generated+1] = np.random.choice(n_ages-30, p=renormalized_adult_older) + 30
            generated_this_step = 2         
                
        # nucleus with other persons (couple, 2 children, adult >= 60)
        elif i == 9:
            for j in range(2):                
                child_age = np.random.choice(30, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+2)]))
            age[num_generated + 2] = mother_current_age
            age[num_generated + 3] = min(n_ages-1,mother_current_age+3)
            age[num_generated + 4] = np.random.choice(n_ages-60, p=renormalized_grandparent) + 60
            generated_this_step = 5
            
        # households with 2 or more nuclei
        # a - couple with same age for mother/father sampled from > 18 + 2 children <= 18 + 2 grand-parents
        # b - couple with same age for mother/father sampled from > 18 + 2 children <= 18 + 2 children from other marriage <= 18
        # scenario b removed for now 
        
        elif i == 10:
            for j in range(2):                
                child_age = np.random.choice(30, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+2)]))
            age[num_generated + 2] = mother_current_age
            age[num_generated + 3] = min(n_ages-1,mother_current_age+3)
            #grandparent_age =  np.random.choice(n_ages-60, p=renormalized_grandparent) + 60
            grandmother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            grandmother_current_age = min(n_ages-1,grandmother_age_at_birth + mother_current_age)
            #age[num_generated + 4] = grandparent_age
            #age[num_generated + 5] = grandparent_age+3   
            age[num_generated + 4] = grandmother_current_age
            age[num_generated + 5] = min(n_ages-1,grandmother_current_age+3)   
            generated_this_step = 6
            
        #elif i == 11:
            #for j in range(4):                
                #child_age = np.random.choice(30, p=renormalized_child)
                #age[num_generated+j] = child_age
            #mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            #mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+4)]))
            #age[num_generated + 4] = mother_current_age
            #age[num_generated + 5] = min(n_ages-1,mother_current_age+3)          
            #generated_this_step = 6
            
        #update list of household contacts
        for i in range(num_generated, num_generated+generated_this_step):
            curr_pos = 0
            for j in range(num_generated, num_generated+generated_this_step):
                if i != j:
                    households[i, curr_pos] = j
                    curr_pos += 1
        num_generated += generated_this_step
        
    return households, age

def sample_households_uk(n):    
    
    max_household_size = 6
    
    households = np.zeros((n, max_household_size), dtype=np.int)
    households[:] = -1
    age = np.zeros(n, dtype=np.int)    
    n_ages = 101
        
    age_distribution = get_age_distribution("United Kingdom")
    age_distribution = np.array(age_distribution)
    age_distribution = age_distribution/age_distribution.sum()
    
    # Source: https://www.statista.com/statistics/813647/share-of-households-uk/        
    # List of household types: 1 retired adult, 2 retired adults, 1 adult without children, 2 adults without children, 
    # 3 or more adults without children (taken equal to 3, possibly different ages),
    # 2 adults with one child, 2 adults with 2 children, 2 adults with 3 or more children (taken equal to 3), 
    # 3 or more adults (a: couple, one age grand parent b: couple, two grand parents) with children (taken equal to 2)
    
    household_probs = np.array([0.14, 0.135, 0.134, 0.212, 0.079, 0.048, 0.083, 0.099, 0.028, 0.021, 0.021])
    household_probs /= household_probs.sum()
    
    num_generated = 0
    
    # from fertility data
    mother_birth_age_distribution=get_mother_birth_age_distribution("UnitedKingdom")    
    renormalized_mother = mother_birth_age_distribution/mother_birth_age_distribution.sum()
    renormalized_adult = age_distribution[18:]
    renormalized_adult = renormalized_adult/renormalized_adult.sum()
    # 18 considered as majority age, maybe should consider that children may still live with parents until 30 or so
    renormalized_child = age_distribution[:18]
    renormalized_child = renormalized_child/renormalized_child.sum()
    # 60 considered as retirement threshold, maybe should be larger, but reasonable for first pass
    renormalized_grandparent = age_distribution[60:]
    renormalized_grandparent = renormalized_grandparent/renormalized_grandparent.sum()
    
    while num_generated < n:
        if n - num_generated < (max_household_size+1):
            i = 0
        else:
            i = np.random.choice(household_probs.shape[0], p=household_probs)
        # retired adult
        if i == 0:
            age[num_generated] = np.random.choice(n_ages-60, p=renormalized_grandparent) + 60
            generated_this_step = 1
        # 2 retired adults
        elif i == 1:  
            age_adult = np.random.choice(n_ages-60, p=renormalized_grandparent) + 60
            age[num_generated] = age_adult
            age[num_generated+1] = min(n_ages-1,age_adult+3) # man three years older on average
            generated_this_step = 2
        # 1 adult without children
        elif i == 2:            
            age[num_generated] = np.random.choice(n_ages-18, p=renormalized_adult) + 18
            generated_this_step = 1      
        # 2 adults without chidren
        elif i == 3:
            age_adult = np.random.choice(n_ages-18, p=renormalized_adult) + 18
            age[num_generated] = age_adult
            age[num_generated+1] = min(n_ages-1,age_adult+3)
            generated_this_step = 2
        # 3 adults without children
        elif i == 4:
            age[num_generated] = np.random.choice(n_ages-18, p=renormalized_adult) + 18
            age[num_generated+1] = np.random.choice(n_ages-18, p=renormalized_adult) + 18
            age[num_generated+2] = np.random.choice(n_ages-18, p=renormalized_adult) + 18
            generated_this_step = 3
        # 1 adult with children (taken equal to 1)        
        elif i == 5:
            child_age = np.random.choice(18, p=renormalized_child)
            age[num_generated] = child_age
            #super rough approximation
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = mother_age_at_birth + child_age
            age[num_generated + 1] = mother_current_age
            generated_this_step = 2
        # 2 adults with one child
        elif i == 6:
            child_age = np.random.choice(18, p=renormalized_child)
            age[num_generated] = child_age
            #super rough approximation
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = mother_age_at_birth + child_age
            age[num_generated + 1] = mother_current_age
            age[num_generated + 2] = min(n_ages-1,mother_current_age+3)
            generated_this_step = 3
        # 2 adults with 2 children
        elif i == 7:
            for j in range(2):                
                child_age = np.random.choice(18, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = mother_age_at_birth + max(age[num_generated:(num_generated+2)])
            age[num_generated + 2] = mother_current_age
            age[num_generated + 3] = min(n_ages-1,mother_current_age+3)
            generated_this_step = 4   
        # 2 adults with 3 children
        elif i == 8:
            for j in range(3):                
                child_age = np.random.choice(18, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = mother_age_at_birth + max(age[num_generated:(num_generated+3)])
            age[num_generated + 3] = mother_current_age
            age[num_generated + 4] = min(n_ages-1,mother_current_age+3)
            generated_this_step = 5
        # 3 adults + children - a
        elif i == 9:
            for j in range(2):                
                child_age = np.random.choice(18, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = mother_age_at_birth + max(age[num_generated:(num_generated+2)])
            age[num_generated + 2] = mother_current_age
            age[num_generated + 3] = min(n_ages-1,mother_current_age+3)
            grandparent_age =  np.random.choice(n_ages-60, p=renormalized_grandparent) + 60
            age[num_generated + 4] = grandparent_age  
            generated_this_step = 5
        # 3 adults + children - b
        elif i == 10:
            for j in range(2):                
                child_age = np.random.choice(18, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = mother_age_at_birth + max(age[num_generated:(num_generated+2)])
            age[num_generated + 2] = mother_current_age
            age[num_generated + 3] = min(n_ages-1,mother_current_age+3)
            #grandparent_age =  np.random.choice(n_ages-60, p=renormalized_grandparent) + 60
            grandmother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            grandmother_current_age = grandmother_age_at_birth + mother_current_age
            #age[num_generated + 4] = grandparent_age
            #age[num_generated + 5] = grandparent_age+3   
            age[num_generated + 4] = grandmother_current_age
            age[num_generated + 5] = min(n_ages-1,grandmother_current_age+3) 
            generated_this_step = 6
            
        #update list of household contacts
        for i in range(num_generated, num_generated+generated_this_step):
            curr_pos = 0
            for j in range(num_generated, num_generated+generated_this_step):
                if i != j:
                    households[i, curr_pos] = j
                    curr_pos += 1
        num_generated += generated_this_step
        
    return households, age

def sample_households_spain(n):
    
    max_household_size = 6
    
    households = np.zeros((n, max_household_size), dtype=np.int)
    households[:] = -1
    age = np.zeros(n, dtype=np.int)    
    n_ages = 101
        
    age_distribution = get_age_distribution("Spain")
    age_distribution = np.array(age_distribution)
    age_distribution = age_distribution/age_distribution.sum()
    
    # Source: https://www.ine.es/en/prensa/np837_en.pdf
    # List of household types: single person, couple without children living in the household, couple with 1 child, couple with 2 children,
    # couple with 3 children or more (taken equal to 3), one adult with children (with 1, 2, 3),
    # couple or father/mother with children and other persons (couple with 2 children, person aged >= 60)
    # more than one family nucleus, others (approximated by family without nucleus - 2 adult ages)
    household_probs = np.array([0.242, 0.216, 0.164, 0.153, 0.032, 0.047, 0.0282, 0.0188, 0.026, 0.021, 0.052])
    household_probs /= household_probs.sum()
    
    num_generated = 0
    
    # from fertility data
    mother_birth_age_distribution=get_mother_birth_age_distribution("Spain")    
    renormalized_mother = mother_birth_age_distribution/mother_birth_age_distribution.sum()
    renormalized_adult = age_distribution[18:]
    renormalized_adult = renormalized_adult/renormalized_adult.sum()
    # 18 considered as majority age, maybe should consider that children may still live with parents until 30 or so
    renormalized_child = age_distribution[:18]
    renormalized_child = renormalized_child/renormalized_child.sum()
    # 60 considered as retirement threshold, maybe should be larger, but reasonable for first pass
    renormalized_grandparent = age_distribution[60:]
    renormalized_grandparent = renormalized_grandparent/renormalized_grandparent.sum()
    
    while num_generated < n:
        if n - num_generated < (max_household_size+1):
            i = 0
        else:
            i = np.random.choice(household_probs.shape[0], p=household_probs) 
            
        # single person
        if i == 0:
            age[num_generated] = np.random.choice(n_ages-18, p=renormalized_adult) + 18
            generated_this_step = 1            
        # couple without children
        elif i == 1:
            age_adult = np.random.choice(n_ages-18, p=renormalized_adult) + 18
            age[num_generated] = age_adult
            age[num_generated+1] = min(n_ages-1,age_adult+3)
            generated_this_step = 2            
        # couple with 1 child
        elif i == 2:
            child_age = np.random.choice(18, p=renormalized_child)
            age[num_generated] = child_age
            #super rough approximation
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = mother_age_at_birth + child_age
            age[num_generated + 1] = mother_current_age
            age[num_generated + 2] = min(n_ages-1,mother_current_age+3)
            generated_this_step = 3            
        # couple with 2 children
        elif i == 3:
            for j in range(2):                
                child_age = np.random.choice(18, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = mother_age_at_birth + max(age[num_generated:(num_generated+2)])
            age[num_generated + 2] = mother_current_age
            age[num_generated + 3] = min(n_ages-1,mother_current_age+3)
            generated_this_step = 4              
        # couple with 3 children or more 
        elif i == 4:
            for j in range(3):                
                child_age = np.random.choice(18, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = mother_age_at_birth + max(age[num_generated:(num_generated+3)])
            age[num_generated + 3] = mother_current_age
            age[num_generated + 4] = min(n_ages-1,mother_current_age+3)
            generated_this_step = 5
        # one adult with children
        # 1 child (50%)
        elif i == 5:
            child_age = np.random.choice(18, p=renormalized_child)
            age[num_generated] = child_age
            #super rough approximation
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = mother_age_at_birth + child_age
            age[num_generated + 1] = mother_current_age
            generated_this_step = 2
        # 2 children (30%)
        elif i == 6:
            for j in range(2):                
                child_age = np.random.choice(18, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = mother_age_at_birth + max(age[num_generated:(num_generated+2)])
            age[num_generated + 2] = mother_current_age
            generated_this_step = 3
        # 3 children (20%)
        elif i == 7:
            for j in range(3):                
                child_age = np.random.choice(18, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = mother_age_at_birth + max(age[num_generated:(num_generated+3)])
            age[num_generated + 3] = mother_current_age
            generated_this_step = 4
    
        # couple or father/mother with children and other persons
        elif i == 8:
            for j in range(2):                
                child_age = np.random.choice(18, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = mother_age_at_birth + max(age[num_generated:(num_generated+2)])
            age[num_generated + 2] = mother_current_age
            age[num_generated + 3] = min(n_ages-1,mother_current_age+3)
            age[num_generated + 4] = np.random.choice(n_ages-60, p=renormalized_grandparent) + 60
            generated_this_step = 5
    
        # similar to "more than one family nucleus" in Italy
        # approximated by: a - couple with same age for mother/father sampled from > 18 + 2 children <= 18 + 2 grand-parents
        elif i == 9:
            for j in range(2):                
                child_age = np.random.choice(18, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = mother_age_at_birth + max(age[num_generated:(num_generated+2)])
            age[num_generated + 2] = mother_current_age
            age[num_generated + 3] = min(n_ages-1,mother_current_age+3)
            #grandparent_age =  np.random.choice(n_ages-60, p=renormalized_grandparent) + 60
            grandmother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            grandmother_current_age = grandmother_age_at_birth + mother_current_age
            #age[num_generated + 4] = grandparent_age
            #age[num_generated + 5] = grandparent_age+3   
            age[num_generated + 4] = grandmother_current_age
            age[num_generated + 5] = min(n_ages-1,grandmother_current_age+3)  
            generated_this_step = 6

        # others
        elif i == 10:
            age[num_generated] = np.random.choice(n_ages-18, p=renormalized_adult) + 18
            age[num_generated+1] = np.random.choice(n_ages-18, p=renormalized_adult) + 18
            generated_this_step = 2   
            
        #update list of household contacts
        for i in range(num_generated, num_generated+generated_this_step):
            curr_pos = 0
            for j in range(num_generated, num_generated+generated_this_step):
                if i != j:
                    households[i, curr_pos] = j
                    curr_pos += 1
        num_generated += generated_this_step
        
    return households, age

def sample_households_germany(n): 
        
    max_household_size = 6
    
    households = np.zeros((n, max_household_size), dtype=np.int)
    households[:] = -1
    age = np.zeros(n, dtype=np.int)    
    n_ages = 101
    
    age_distribution = get_age_distribution("Germany")
    age_distribution = np.array(age_distribution)
    age_distribution = age_distribution/age_distribution.sum()
    
    # Source 1: https://www.statista.com/statistics/464187/households-by-size-germany/
    # Source 2 (3 tables): https://www.destatis.de/EN/Themes/Society-Environment/Population/Households-Families/Tables/couples.html, 
    # https://www.destatis.de/EN/Themes/Society-Environment/Population/Households-Families/Tables/families.html,
    # https://www.destatis.de/EN/Themes/Society-Environment/Population/Households-Families/Tables/unattached-people.html
    
    age_18_24 = age_distribution[18:25]
    age_18_24 /= age_18_24.sum()
    age_25_44 = age_distribution[25:45]
    age_25_44 /= age_25_44.sum()
    age_45_64 = age_distribution[45:65]
    age_45_64 /= age_45_64.sum()
    age_65_84 = age_distribution[65:85]
    age_65_84 /= age_65_84.sum()
    age_85 = age_distribution[85:]
    age_85 /= age_85.sum()
    
    age_one_person_scaling = np.array([0.0761527, 0.280135, 0.297618, 0.282802, 0.0632923])
    age_one_person_scaling *= 0.4189042
    
    two_person_scaling = np.array([0.862261, 0.106558, 0.031181])
    two_person_scaling *= 0.337941
    
    age_child_0_17 = age_distribution[:18]
    age_child_0_17 /= age_child_0_17.sum()
    age_child_0_17_final = np.zeros(30)
    age_child_0_17_final[:18] = age_child_0_17
    age_child_18_30 = age_distribution[18:30]
    age_child_18_30 /= age_child_18_30.sum()
    age_child_18_30_final = np.zeros(30)
    age_child_18_30_final[18:] = age_child_18_30
    
    renormalized_child = 0.741 * age_child_0_17_final + 0.259 * age_child_18_30_final
    
    renormalized_adult = age_distribution[18:]
    renormalized_adult /= renormalized_adult.sum()
    renormalized_grandparent = age_distribution[60:]
    renormalized_grandparent /= renormalized_grandparent.sum()
    
    # from fertility data
    mother_birth_age_distribution=get_mother_birth_age_distribution("Germany")    
    renormalized_mother = mother_birth_age_distribution/mother_birth_age_distribution.sum()
    
    # 5 scenarios, 3 scenarios, 2 scenarios, 3 scenarios
    household_probs = np.concatenate([age_one_person_scaling, age_two_person_scaling,0.118979,0.0905817,
                                      0.0167968,0.00839838,0.00839838])
    household_probs /= household_probs.sum()
 
    num_generated = 0
    
    
    while num_generated < n:
        if n - num_generated < (max_household_size+1):
            i = 0
        else:
            i = np.random.choice(household_probs.shape[0], p=household_probs) 
            
        # single person - 18-24
        if i == 0:
            age[num_generated] = np.random.choice(24-18+1, p=age_18_24) + 18
            generated_this_step = 1            
        # single person - 25-44
        elif i == 1:
            age[num_generated] = np.random.choice(44-25+1, p=age_25_44) + 25
            generated_this_step = 1       
        # single person - 45-64
        elif i == 2:
            age[num_generated] = np.random.choice(64-45+1, p=age_45_64) + 45
            generated_this_step = 1             
        # single person - 65-84
        elif i == 3:
            age[num_generated] = np.random.choice(84-65+1, p=age_65_84) + 65
            generated_this_step = 1            
        # single person - >= 85
        elif i == 4:
            age[num_generated] = np.random.choice(n_ages-85, p=age_85) + 85
            generated_this_step = 1  
            
        # couple without children
        elif i == 5:
            age_adult = np.random.choice(n_ages-18, p=renormalized_adult) + 18
            age[num_generated] = age_adult
            age[num_generated+1] = min(n_ages-1,age_adult+3)
            generated_this_step = 2  
            
        # lone parent + one child
        elif i == 6:
            child_age = np.random.choice(18, p=renormalized_child)
            age[num_generated] = child_age
            #super rough approximation
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = mother_age_at_birth + child_age
            age[num_generated + 1] = mother_current_age
            generated_this_step = 2
            
        # other - 2 random ages
        elif i == 7:
            age[num_generated] = np.random.choice(n_ages-18, p=renormalized_adult) + 18
            age[num_generated+1] = np.random.choice(n_ages-18, p=renormalized_adult) + 18
            generated_this_step = 2
            
        # couple with 1 child
        elif i == 8:
            child_age = np.random.choice(18, p=renormalized_child)
            age[num_generated] = child_age
            #super rough approximation
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = mother_age_at_birth + child_age
            age[num_generated + 1] = mother_current_age
            age[num_generated + 2] = min(n_ages-1,mother_current_age+3)
            generated_this_step = 3   
            
        # couple + 2 children
        elif i == 9:
            for j in range(2):                
                child_age = np.random.choice(30, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = mother_age_at_birth + max(age[num_generated:(num_generated+2)])
            age[num_generated + 2] = mother_current_age
            age[num_generated + 3] = min(n_ages-1,mother_current_age+3)
            generated_this_step = 4          
        
        # couple + 3 children
        elif i == 10:
            for j in range(3):                
                child_age = np.random.choice(18, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = mother_age_at_birth + max(age[num_generated:(num_generated+3)])
            age[num_generated + 3] = mother_current_age
            age[num_generated + 4] = min(n_ages-1,mother_current_age+3)
            generated_this_step = 5
        
        # couple + 4 children
        elif i == 11:
            for j in range(4):                
                child_age = np.random.choice(18, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = mother_age_at_birth + max(age[num_generated:(num_generated+4)])
            age[num_generated + 4] = mother_current_age
            age[num_generated + 5] = min(n_ages-1,mother_current_age+3)
            generated_this_step = 6
        
        # couple + 2 children + grand-mother      
        elif i == 12:
            for j in range(2):                
                child_age = np.random.choice(18, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = mother_age_at_birth + max(age[num_generated:(num_generated+2)])
            age[num_generated + 2] = mother_current_age
            age[num_generated + 3] = min(n_ages-1,mother_current_age+3)
            #grandparent_age =  np.random.choice(n_ages-60, p=renormalized_grandparent) + 60
            grandmother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            grandmother_current_age = grandmother_age_at_birth + mother_current_age
            #age[num_generated + 4] = grandparent_age
            #age[num_generated + 5] = grandparent_age+3   
            age[num_generated + 4] = grandmother_current_age
            generated_this_step = 5
            
        #update list of household contacts
        for i in range(num_generated, num_generated+generated_this_step):
            curr_pos = 0
            for j in range(num_generated, num_generated+generated_this_step):
                if i != j:
                    households[i, curr_pos] = j
                    curr_pos += 1
        num_generated += generated_this_step
        
    return households, age
"""

def sample_households_maharashtra(n):
    print("LOL3")
    max_household_size = 9
    son_lives_with_parents_till_age=18
    grandparent_age=60
    #parent_uppercap=50
    households = np.zeros((n, max_household_size), dtype=np.int)
    households[:] = -1
    age = np.zeros(n, dtype=np.int)    
    n_ages = 101
    age_distribution= [1.63, 1.54, 1.73, 1.67, 1.68, 1.50, 1.60, 1.58, 1.67, 1.54, 1.67, 1.59, 1.78, 1.65, 2.01, 1.76,
                       1.65, 1.76, 1.93, 1.68, 1.98, 1.84, 2.14, 1.83, 1.95, 2.29, 1.91, 1.71, 1.91, 1.50, 2.23, 1.20,
                       1.78, 1.14, 1.24, 2.20, 1.19, 1.17, 1.37, 1.18, 2.01 ,0.94, 1.38, 1.03, 0.91 ,1.86, 0.95, 0.92,
                       0.99, 0.76, 1.78, 0.79, 1.05, 0.66, 0.73, 1.66, 0.70, 0.66, 0.71, 0.50, 1.74, 0.44, 0.70, 0.45,
                       0.45, 1.84, 0.44, 0.51, 0.41, 0.36, 1.12, 0.22, 0.37, 0.18, 0.20, 0.64, 0.12, 0.12 ,0.14, 0.07,
                       0.38, 0.06, 0.10, 0.05, 0.07, 0.15, 0.03, 0.03, 0.02, 0.03, 0.10, 0.00,0.01, 0.01, 0.01, 0.07,0.02, 0,0,0,0]    
    age_distribution = np.array(age_distribution)
    age_distribution = age_distribution/age_distribution.sum()
        
    # List of household types: single household, couple without children, single parent +1/2/3 children, couple +1/2/3 children,
    # family without a nucleus, nucleus with other persons, households with two or more nuclei (a and b)
    #household_probs = np.array([0.308179, 0.191000, 0.0694283, 0.0273065, 0.00450268, 0.152655, 0.132429, 0.0340969, 
    #                  0.043821, 0.033, 0.0150])
    
    household_probs= np.array([0.0394284038, 0.1135717199, 0.02265637467 ,0.04265637467, 0.006669284629,0.1135717199 ,0.2117641302 ,0.2117641302 ,
                               0.009533560241, 0.123714170745, 0.104670131])
    
    household_probs /= household_probs.sum()
    
    num_generated = 0
    
    # from fertility data
    mother_birth_age_distribution=get_mother_birth_age_distribution("India")    
    renormalized_mother = mother_birth_age_distribution/mother_birth_age_distribution.sum()
    renormalized_adult = age_distribution[18:]
    renormalized_adult = renormalized_adult/renormalized_adult.sum()
    # 18 considered as majority age, maybe should consider that children may still live with parents until 30 or so
    renormalized_child = age_distribution[:son_lives_with_parents_till_age]
    renormalized_child = renormalized_child/renormalized_child.sum()
    
    renormalized_adult_older = age_distribution[son_lives_with_parents_till_age:]
    renormalized_adult_older /= renormalized_adult_older.sum()
    # grandparent_age considered as retirement threshold, maybe should be larger, but reasonable for first pass
    renormalized_grandparent = age_distribution[grandparent_age:]
    renormalized_grandparent = renormalized_grandparent/renormalized_grandparent.sum()
    
    while num_generated < n:
        if n - num_generated < (max_household_size+1):
            i = 0
        else:
            i = np.random.choice(household_probs.shape[0], p=household_probs)
        #single person household
        #sample from age distribution
        if i == 0:
            age[num_generated] = np.random.choice(n_ages-son_lives_with_parents_till_age, p=renormalized_adult_older) + son_lives_with_parents_till_age
            generated_this_step = 1
        # couple, sample from age distribution conditioned on age >= 18
        elif i == 1:  
            age_adult = np.random.choice(n_ages-son_lives_with_parents_till_age, p=renormalized_adult_older) + son_lives_with_parents_till_age
            age[num_generated] = age_adult
            age[num_generated+1] = min(n_ages-1,age_adult+3) # man three years older on average
            generated_this_step = 2
        # single parent, 1 child
        elif i == 2:            
            child_age = np.random.choice(son_lives_with_parents_till_age, p=renormalized_child)
            age[num_generated] = child_age
            #super rough approximation
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + child_age)
            age[num_generated + 1] = mother_current_age
            generated_this_step = 2
        # single parent, 2 children
        elif i == 3:
            for j in range(2):                
                child_age = np.random.choice(son_lives_with_parents_till_age, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+2)]))
            age[num_generated + 2] = mother_current_age
            generated_this_step = 3
        # single parent, 3 children
        elif i == 4:
            for j in range(3):                
                child_age = np.random.choice(son_lives_with_parents_till_age, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+3)]))
            age[num_generated + 3] = mother_current_age
            generated_this_step = 4
            
        # couple, 1 child
        elif i == 5: 
            child_age = np.random.choice(son_lives_with_parents_till_age, p=renormalized_child)
            age[num_generated] = child_age
            #super rough approximation
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + child_age)
            age[num_generated + 1] = mother_current_age
            age[num_generated + 2] = min(n_ages-1,mother_current_age+3)
            generated_this_step = 3
        
        # couple, 2 children
        elif i == 6:
            for j in range(2):                
                child_age = np.random.choice(son_lives_with_parents_till_age, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+2)]))
            age[num_generated + 2] = mother_current_age
            age[num_generated + 3] = min(n_ages-1,mother_current_age+3)
            generated_this_step = 4            
        
        # couple, 3 children
        elif i == 7:
            for j in range(3):                
                child_age = np.random.choice(son_lives_with_parents_till_age, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+3)]))
            age[num_generated + 3] = mother_current_age
            age[num_generated + 4] = min(n_ages-1,mother_current_age+3)
            generated_this_step = 5
        
        # family without nucleus
        elif i == 8:
            age[num_generated] = np.random.choice(n_ages-son_lives_with_parents_till_age, p=renormalized_adult_older) + son_lives_with_parents_till_age
            age[num_generated+1] = np.random.choice(n_ages-son_lives_with_parents_till_age, p=renormalized_adult_older) + son_lives_with_parents_till_age
            generated_this_step = 2         
                
        # nucleus with other persons (couple, 2 children, adult >= 60)
        elif i == 9:
            for j in range(2):                
                child_age = np.random.choice(son_lives_with_parents_till_age, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+2)]))
            age[num_generated + 2] = mother_current_age
            age[num_generated + 3] = min(n_ages-1,mother_current_age+3)
            age[num_generated + 4] = np.random.choice(n_ages-18, p=renormalized_adult) + 18
            generated_this_step = 5
            
        # households with 2 or more nuclei
        # a - couple with same age for mother/father sampled from > 18 + 2 children <= 18 + 2 grand-parents
        # b - couple with same age for mother/father sampled from > 18 + 2 children <= 18 + 2 children from other marriage <= 18
        # scenario b removed for now 
        
        elif i == 10:
            for j in range(4):                
                child_age = np.random.choice(son_lives_with_parents_till_age, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+4)]))
            age[num_generated + 4] = mother_current_age
            age[num_generated + 5] = min(n_ages-1,mother_current_age+3)
            #grandparent_age =  np.random.choice(n_ages-60, p=renormalized_grandparent) + 60
            grandmother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            grandmother_current_age = min(n_ages-1,grandmother_age_at_birth + mother_current_age)
            #age[num_generated + 4] = grandparent_age
            #age[num_generated + 5] = grandparent_age+3   
            age[num_generated + 5] = grandmother_current_age
            age[num_generated + 6] = min(n_ages-1,grandmother_current_age+3)   
            generated_this_step = 8
            
        #elif i == 11:
            #for j in range(4):                
                #child_age = np.random.choice(30, p=renormalized_child)
                #age[num_generated+j] = child_age
            #mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            #mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+4)]))
            #age[num_generated + 4] = mother_current_age
            #age[num_generated + 5] = min(n_ages-1,mother_current_age+3)          
            #generated_this_step = 6
            
        #update list of household contacts
        for i in range(num_generated, num_generated+generated_this_step):
            curr_pos = 0
            for j in range(num_generated, num_generated+generated_this_step):
                if i != j:
                    households[i, curr_pos] = j
                    curr_pos += 1
        num_generated += generated_this_step
        
    return households, age

In [122]:
n_ages = 101
age_distribution= [2.1,2.03,2.03,2.14,2.08,2.12,2.31,2.22,2.47,2.01,2.76,2.02,2.69,2.49,2.2,2.45,2.41,2.15,
                       2.75,1.72,2.46,1.55,2.19,1.61,1.55,2.23,1.56,1.23,1.8,0.81,2.25,0.67,1.56,0.8,0.79,2.26,
                       0.98,0.73,1.25,0.58,1.96,0.54,1.16,0.64,0.57,1.87,0.72,0.61,0.96,0.48,1.08,0.61,1.02,0.56,
                       0.54,1.62,0.65,0.42,0.7,0.28,1.73,0.29,0.65,0.27,0.23,1.52,0.2,0.19,0.31,0.11,1.1,0.09,0.25,
                       0.09,0.08,0.58,0.09,0.04,0.09,0.03,0.42,0.03,0.07,0.02,0.02,0.19,0.02,0.02,0.02,0.01,0.1,0,
                       0.01,0,0,0.03,0.01,0.01,0.01,0.01,0.01]
def sample_households_up(n):
    print("LOL3_up")
    max_household_size = 10
    son_lives_with_parents_till_age=30
    
    households = np.zeros((n, max_household_size), dtype=np.int)
    households[:] = -1
    age = np.zeros(n, dtype=np.int)    
    n_ages = 101
    age_distribution= [2.1,2.03,2.03,2.14,2.08,2.12,2.31,2.22,2.47,2.01,2.76,2.02,2.69,2.49,2.2,2.45,2.41,2.15,
                       2.75,1.72,2.46,1.55,2.19,1.61,1.55,2.23,1.56,1.23,1.8,0.81,2.25,0.67,1.56,0.8,0.79,2.26,
                       0.98,0.73,1.25,0.58,1.96,0.54,1.16,0.64,0.57,1.87,0.72,0.61,0.96,0.48,1.08,0.61,1.02,0.56,
                       0.54,1.62,0.65,0.42,0.7,0.28,1.73,0.29,0.65,0.27,0.23,1.52,0.2,0.19,0.31,0.11,1.1,0.09,0.25,
                       0.09,0.08,0.58,0.09,0.04,0.09,0.03,0.42,0.03,0.07,0.02,0.02,0.19,0.02,0.02,0.02,0.01,0.1,0,
                       0.01,0,0,0.03,0.01,0.01,0.01,0.01,0.01]
    age_distribution = np.array(age_distribution)
    age_distribution = age_distribution/age_distribution.sum()
        
    # List of household types: single household, couple without children, single parent +1/2/3 children, couple +1/2/3 children,
    # family without a nucleus, nucleus with other persons, households with two or more nuclei (a and b)
    #household_probs = np.array([0.308179, 0.191000, 0.0694283, 0.0273065, 0.00450268, 0.152655, 0.132429, 0.0340969, 
    #                  0.043821, 0.033, 0.0150])
    
    household_probs= np.array([0.0594284038, 0.1135717199, 0.02265637467 ,0.02265637467, 0.006669284629,0.1135717199 ,0.2117641302 ,0.2117641302 ,
                               0.009533560241, 0.003714170745, 0.224670131])
    
    household_probs /= household_probs.sum()
    
    num_generated = 0
    
    # from fertility data
    mother_birth_age_distribution=get_mother_birth_age_distribution("India")    
    renormalized_mother = mother_birth_age_distribution/mother_birth_age_distribution.sum()
    renormalized_adult = age_distribution[18:]
    renormalized_adult = renormalized_adult/renormalized_adult.sum()
    # 18 considered as majority age, maybe should consider that children may still live with parents until 30 or so
    renormalized_child = age_distribution[:30]
    renormalized_child = renormalized_child/renormalized_child.sum()
    
    renormalized_adult_older = age_distribution[30:]
    renormalized_adult_older /= renormalized_adult_older.sum()
    # 60 considered as retirement threshold, maybe should be larger, but reasonable for first pass
    renormalized_grandparent = age_distribution[60:]
    renormalized_grandparent = renormalized_grandparent/renormalized_grandparent.sum()
    
    while num_generated < n:
        if n - num_generated < (max_household_size+1):
            i = 0
        else:
            i = np.random.choice(household_probs.shape[0], p=household_probs)
        #single person household
        #sample from age distribution
        if i == 0:
            age[num_generated] = np.random.choice(n_ages-30, p=renormalized_adult_older) + 30
            generated_this_step = 1
        # couple, sample from age distribution conditioned on age >= 18
        elif i == 1:  
            age_adult = np.random.choice(n_ages-30, p=renormalized_adult_older) + 30
            age[num_generated] = age_adult
            age[num_generated+1] = min(n_ages-1,age_adult+3) # man three years older on average
            generated_this_step = 2
        # single parent, 1 child
        elif i == 2:            
            child_age = np.random.choice(30, p=renormalized_child)
            age[num_generated] = child_age
            #super rough approximation
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + child_age)
            age[num_generated + 1] = mother_current_age
            generated_this_step = 2
        # single parent, 2 children
        elif i == 3:
            for j in range(2):                
                child_age = np.random.choice(30, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+2)]))
            age[num_generated + 2] = mother_current_age
            generated_this_step = 3
        # single parent, 3 children
        elif i == 4:
            for j in range(3):                
                child_age = np.random.choice(30, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+3)]))
            age[num_generated + 3] = mother_current_age
            generated_this_step = 4
            
        # couple, 1 child
        elif i == 5: 
            child_age = np.random.choice(30, p=renormalized_child)
            age[num_generated] = child_age
            #super rough approximation
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + child_age)
            age[num_generated + 1] = mother_current_age
            age[num_generated + 2] = min(n_ages-1,mother_current_age+3)
            generated_this_step = 3
        
        # couple, 2 children
        elif i == 6:
            for j in range(2):                
                child_age = np.random.choice(30, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+2)]))
            age[num_generated + 2] = mother_current_age
            age[num_generated + 3] = min(n_ages-1,mother_current_age+3)
            generated_this_step = 4            
        
        # couple, 3 children
        elif i == 7:
            for j in range(3):                
                child_age = np.random.choice(30, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+3)]))
            age[num_generated + 3] = mother_current_age
            age[num_generated + 4] = min(n_ages-1,mother_current_age+3)
            generated_this_step = 5
        
        # family without nucleus
        elif i == 8:
            age[num_generated] = np.random.choice(n_ages-30, p=renormalized_adult_older) + 30
            age[num_generated+1] = np.random.choice(n_ages-30, p=renormalized_adult_older) + 30
            generated_this_step = 2         
                
        # nucleus with other persons (couple, 2 children, adult >= 60)
        elif i == 9:
            for j in range(2):                
                child_age = np.random.choice(30, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+2)]))
            age[num_generated + 2] = mother_current_age
            age[num_generated + 3] = min(n_ages-1,mother_current_age+3)
            age[num_generated + 4] = np.random.choice(n_ages-60, p=renormalized_grandparent) + 60
            generated_this_step = 5
            
        # households with 2 or more nuclei
        # a - couple with same age for mother/father sampled from > 18 + 2 children <= 18 + 2 grand-parents
        # b - couple with same age for mother/father sampled from > 18 + 2 children <= 18 + 2 children from other marriage <= 18
        # scenario b removed for now 
        
        elif i == 10:
            for j in range(2):                
                child_age = np.random.choice(30, p=renormalized_child)
                age[num_generated+j] = child_age
            mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+2)]))
            age[num_generated + 2] = mother_current_age
            age[num_generated + 3] = min(n_ages-1,mother_current_age+3)
            #grandparent_age =  np.random.choice(n_ages-60, p=renormalized_grandparent) + 60
            grandmother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            grandmother_current_age = min(n_ages-1,grandmother_age_at_birth + mother_current_age)
            #age[num_generated + 4] = grandparent_age
            #age[num_generated + 5] = grandparent_age+3   
            age[num_generated + 4] = grandmother_current_age
            age[num_generated + 5] = min(n_ages-1,grandmother_current_age+3)   
            generated_this_step = 6
            
        #elif i == 11:
            #for j in range(4):                
                #child_age = np.random.choice(30, p=renormalized_child)
                #age[num_generated+j] = child_age
            #mother_age_at_birth = (np.random.choice(7, p=renormalized_mother) + 3)*5+np.random.randint(5)
            #mother_current_age = min(n_ages-1,mother_age_at_birth + max(age[num_generated:(num_generated+4)]))
            #age[num_generated + 4] = mother_current_age
            #age[num_generated + 5] = min(n_ages-1,mother_current_age+3)          
            #generated_this_step = 6
            
        #update list of household contacts
        for i in range(num_generated, num_generated+generated_this_step):
            curr_pos = 0
            for j in range(num_generated, num_generated+generated_this_step):
                if i != j:
                    households[i, curr_pos] = j
                    curr_pos += 1
        num_generated += generated_this_step
        
    return households, age

In [123]:
n = 10000000
#from sample_households import sample_households_maharashtra
#from sample_households import sample_households_up
#households, age = sample_households_italy(10000000)
#households, age = sample_households_maharashtra(10000000)
households, age = sample_households_up(10000000)

LOL3_up


In [124]:
#age=np.array([int(item*(31./37.)) for item in age])
# FOR MAHARASHTRA
'''
for i, item in enumerate(age):
    if item>60 and np.random.uniform()>=0.5:
        age[i]=np.random.uniform(101)
'''

# FOR UP
age_distribution=np.array(age_distribution)
for i, item in enumerate(age):
    if item>50 and item <70 and np.random.uniform()>=0.6:
        age[i]=np.random.choice(n_ages-71, p=age_distribution[:30]/sum(age_distribution[:30]))
    elif item >=70 and np.random.uniform()>=0.4:
        age[i]=np.random.choice(n_ages-71, p=age_distribution[:30]/sum(age_distribution[:30]))


In [125]:
age.mean()

29.2048251

In [126]:
age.max()

100

In [127]:
import numpy as np
import csv
import math
y = np.bincount(age)
age_ratios = y/y.sum()

In [128]:
age_0_14 = age_ratios[0:15].sum()
age_15_29 = age_ratios[15:30].sum()
age_30_49 = age_ratios[30:50].sum()
age_50_69 = age_ratios[50:70].sum()
age_70_plus = age_ratios[70:].sum()

In [129]:
print('Sampled')
print('0-14: ', age_0_14)
print('15-29: ', age_15_29)
print('30-49: ', age_30_49)
print('50-69: ', age_50_69)
print('70+: ', age_70_plus)

Sampled
0-14:  0.2978214
15-29:  0.26573990000000003
30-49:  0.2683261
50-69:  0.1354216
70+:  0.032691


In [130]:
age_distribution_up= [2.1,2.03,2.03,2.14,2.08,2.12,2.31,2.22,2.47,2.01,2.76,2.02,2.69,2.49,2.2,2.45,2.41,2.15,
                       2.75,1.72,2.46,1.55,2.19,1.61,1.55,2.23,1.56,1.23,1.8,0.81,2.25,0.67,1.56,0.8,0.79,2.26,
                       0.98,0.73,1.25,0.58,1.96,0.54,1.16,0.64,0.57,1.87,0.72,0.61,0.96,0.48,1.08,0.61,1.02,0.56,
                       0.54,1.62,0.65,0.42,0.7,0.28,1.73,0.29,0.65,0.27,0.23,1.52,0.2,0.19,0.31,0.11,1.1,0.09,0.25,
                       0.09,0.08,0.58,0.09,0.04,0.09,0.03,0.42,0.03,0.07,0.02,0.02,0.19,0.02,0.02,0.02,0.01,0.1,0,
                       0.01,0,0,0.03,0.01,0.01,0.01,0.01,0.01]
age_distribution_maharashtra=[1.63, 1.54, 1.73, 1.67, 1.68, 1.50, 1.60, 1.58, 1.67, 1.54, 1.67, 1.59, 1.78, 1.65, 2.01, 1.76,
                       1.65, 1.76, 1.93, 1.68, 1.98, 1.84, 2.14, 1.83, 1.95, 2.29, 1.91, 1.71, 1.91, 1.50, 2.23, 1.20,
                       1.78, 1.14, 1.24, 2.20, 1.19, 1.17, 1.37, 1.18, 2.01 ,0.94, 1.38, 1.03, 0.91 ,1.86, 0.95, 0.92,
                       0.99, 0.76, 1.78, 0.79, 1.05, 0.66, 0.73, 1.66, 0.70, 0.66, 0.71, 0.50, 1.74, 0.44, 0.70, 0.45,
                       0.45, 1.84, 0.44, 0.51, 0.41, 0.36, 1.12, 0.22, 0.37, 0.18, 0.20, 0.64, 0.12, 0.12 ,0.14, 0.07,
                       0.38, 0.06, 0.10, 0.05, 0.07, 0.15, 0.03, 0.03, 0.02, 0.03, 0.10, 0.00,0.01, 0.01, 0.01, 0.07,0.02, 0,0,0,0]

italian_ages = np.array(age_distribution_up)
italian_ages = italian_ages/italian_ages.sum()

In [131]:
italian_ages_0_14 = italian_ages[0:15].sum()
italian_ages_15_29 = italian_ages[15:30].sum()
italian_ages_30_49 = italian_ages[30:50].sum()
italian_ages_50_69 = italian_ages[50:70].sum()
italian_ages_70_plus = italian_ages[70:].sum()

In [132]:
print('Actual')
print('0-14: ', italian_ages_0_14)
print('15-29: ', italian_ages_15_29)
print('30-49: ', italian_ages_30_49)
print('50-69: ', italian_ages_50_69)
print('70+: ', italian_ages_70_plus)

Actual
0-14:  0.3368684342171085
15-29:  0.28484242121060527
30-49:  0.21390695347673833
50-69:  0.1298649324662331
70+:  0.03451725862931464


In [133]:
import numpy as np
print ("LOL")
def p_diabetes_china(age):
    #diabetes prevalence for china
    #https://jamanetwork.com/journals/jama/fullarticle/1734701
    male = np.array([5.2755905511811, 8.346456692913385, 13.543307086614174, 17.95275590551181, 20.708661417322837, 21.653543307086615])
    female = np.array([4.015748031496061, 5.1181102362204705, 9.05511811023622, 17.401574803149607, 24.488188976377955, 25.196850393700785])
    male = male/100
    female = female/100

    #https://www.statista.com/statistics/282119/china-sex-ratio-by-age-group/
    sex_ratio = np.zeros(101)
    sex_ratio[0:5] = 113.91
    sex_ratio[5:10] = 118.03
    sex_ratio[10:15] = 118.62
    sex_ratio[15:20] = 118.14
    sex_ratio[20:25] = 112.89
    sex_ratio[25:30] = 105.39
    sex_ratio[30:35] = 101.05
    sex_ratio[35:40] = 102.84
    sex_ratio[40:45] = 103.75
    sex_ratio[45:50] = 103.64
    sex_ratio[50:55] = 102.15
    sex_ratio[55:60] = 101.65
    sex_ratio[60:65] = 100.5
    sex_ratio[65:70] = 96.94
    sex_ratio[70:75] = 94.42
    sex_ratio[75:80] = 89.15
    sex_ratio[80:85] = 76.97
    sex_ratio[85:90] = 71.16
    sex_ratio[90:95] = 48.74
    sex_ratio[95:] = 40.07

    #calculate male to female ratio within each age bucket, and use this to combine the male vs female
    #prevalence numbers
    sex_ratio = sex_ratio/(sex_ratio + 100)

    age_distribution = [16113.281,16543.361,16875.302,17118.429,17282.064,17375.527,17408.145,17389.238,17328.13,17234.143,17117.175,16987.122,16850.435,16715.289,16592.73,16484.473,16388.786,16370.261,16460.9,16637.439,16866.861,17182.465,17477.132,17702.896,17928.813,18144.994,18201.129,18841.832,20387.657,22413.391,24308.028,26355.485,27269.657,26400.295,24405.505,22597.72,20719.355,19296.916,18726.536,18750.928,18640.938,18451.511,18716.505,19599.644,20865.548,22101.75,23374.699,24376.638,24907.095,25077.435,25250.357,25414.362,25172.526,24383.003,23225.134,22043.117,20795.729,19608.86,18589.082,17703.703,16743.545,15666.543,14988.213,14917.427,15198.411,15425.124,15749.105,15550.741,14503.063,12921.733,11444.972,9939.85,8651.521,7764.623,7148.723,6478.704,5807.535,5222.027,4729.055,4307.295,3931.038,3608.42,3272.336,2887.659,2481.964,2118.152,1783.88,1480.587,1215.358,983.8,739.561,551.765,453.96,342.463,217.275,145.809,122.178,96.793,69.654,40.759,74.692]
    age_distribution = np.array(age_distribution)
    intervals = [(18, 30), (30, 40), (40, 50), (50, 60), (60, 70), (70, 100)]

    percent_male_intervals = np.zeros(len(intervals))
    for i in range(len(intervals)):
        age_frequency_within_interval = age_distribution[intervals[i][0]:intervals[i][1]]/age_distribution[intervals[i][0]:intervals[i][1]].sum()
        percent_male_intervals[i] = np.dot(age_frequency_within_interval, sex_ratio[intervals[i][0]:intervals[i][1]])

    p_diabetes = male*percent_male_intervals + female*(1-percent_male_intervals)

    p_diabetes_expanded = np.zeros(101)
    p_diabetes_expanded[:intervals[0][0]] = 0
    for i in range(len(intervals)):
        p_diabetes_expanded[intervals[i][0]:intervals[i][1]] = p_diabetes[i]
    p_diabetes_expanded[intervals[-1][1]:] = p_diabetes[-1]

    return p_diabetes_expanded

def p_hypertension_china(age):
    p_hyp_data = np.zeros(101)
    p_hyp_data[:18] = 0
    p_hyp_data[18:25] = 4.0
    p_hyp_data[25:35] = 6.1
    p_hyp_data[35:45] = 15.0
    p_hyp_data[45:55] = 29.6
    p_hyp_data[55:65] = 44.6
    p_hyp_data[65:75] = 55.7
    p_hyp_data[75:] = 60.2

    p_hyp_data = p_hyp_data/100

    return p_hyp_data


def sample_joint(age, p_diabetes, p_hyp):
    #https://www-nature-com.ezp-prod1.hul.harvard.edu/articles/hr201767
    p_hyp_given_diabetes = 0.5
    p_hyp_given_not_diabetes = (p_hyp - p_hyp_given_diabetes*p_diabetes)/(1 - p_diabetes)
    diabetes_status = (np.random.rand(age.shape[0]) < p_diabetes[age]).astype(np.int)
    hyp_status = np.zeros(age.shape[0], dtype=np.int)
    hyp_status[diabetes_status == 1] = np.random.rand((diabetes_status == 1).sum()) < p_hyp_given_diabetes
    hyp_status[diabetes_status == 0] = np.random.rand((diabetes_status == 0).sum()) < p_hyp_given_not_diabetes[age[diabetes_status == 0]]
    return diabetes_status, hyp_status

def sample_joint_comorbidities(age, country='China'):
    """
    Default country is China.
    For other countries pass value for country from {us, Republic of Korea, japan, Spain, italy, uk, France}
    """
    print("LOL2")
    return sample_joint(age, p_comorbidity(country, 'diabetes'), p_comorbidity(country, 'hypertension'))

def p_comorbidity(country, comorbidity, warning=False):

    """
    Input:
        -country: a string input belonging to- {us, Republic of Korea, japan, Spain, italy, uk, France}
        -comorbidity: a string input belonging to- {diabetes, hypertension}
        -warning: optional, If set to True, prints out the underlying assumptions/approximations
    Returns:
        -prevalence, sampled from a prevalence array of size 100, where prevalence[i] is the prevalence rate at age between {i, i+1}
    """

    prevalence = np.zeros(101)
    warning_string= " "

    ######################################  US ################################
    if country=='us':

        warning_string= " 1. Prevalence assumed to be linerly increasing between ages 0-18"

        if comorbidity=='diabetes':
            for i in range(101):
                if i<18:
                    prevalence[i]= 0.042*(i/18.)
                elif i<45:
                    prevalence[i]=0.042
                elif i<65:
                    prevalence[i]=0.175
                else:
                    prevalence[i]=0.268

        elif comorbidity=='hypertension':
            for i in range(101):
                if i<18:
                    prevalence[i]= 0.075*(i/18.)
                elif i<40:
                    prevalence[i]=0.075
                elif i<59:
                    prevalence[i]=0.332
                else:
                    prevalence[i]=0.631

    ######################################  south korea #######################
    if country=='Republic of Korea':

        if comorbidity=='diabetes':
            for i in range(101):
                if i <= 4:
                    prevalence[i] = 0.0002
                elif i <= 9:
                    prevalence[i] = 0.0013
                elif i <= 14:
                    prevalence[i] = 0.0043
                elif i <= 19:
                    prevalence[i] = 0.0085
                elif i <= 24:
                    prevalence[i] = 0.0150
                elif i <= 29:
                    prevalence[i] = 0.0238
                elif i <= 34:
                    prevalence[i] = 0.0347
                elif i <= 39:
                    prevalence[i] = 0.0484
                elif i <= 44:
                    prevalence[i] = 0.0658
                elif i <= 49:
                    prevalence[i] = 0.0893
                elif i <= 54:
                    prevalence[i] = 0.1195
                elif i <= 59:
                    prevalence[i] = 0.1547
                elif i <= 64:
                    prevalence[i] = 0.1933
                elif i <= 69:
                    prevalence[i] = 0.2319
                elif i <= 74:
                    prevalence[i] = 0.2672
                elif i <= 79:
                    prevalence[i] = 0.2945
                elif i <= 84:
                    prevalence[i] = 0.3106
                elif i <= 89:
                    prevalence[i] = 0.3067
                else:
                    prevalence[i] = 0.2855

        elif comorbidity=='hypertension':
            for i in range(101):
                if i<30:
                    prevalence[i]= 0.086*(i/30.)
                elif i<40:
                    prevalence[i]=0.086
                elif i<50:
                    prevalence[i]=0.194
                elif i<60:
                    prevalence[i]=0.34
                elif i< 70:
                    prevalence[i]=0.497
                else:
                    prevalence[i]=0.645
        elif comorbidity=='CVD':
            for i in range(101):
                if i <= 4:
                    prevalence[i] = 0.0004
                elif i <= 9:
                    prevalence[i] = 0.0005
                elif i <= 14:
                    prevalence[i] = 0.0011
                elif i <= 19:
                    prevalence[i] = 0.0028
                elif i <= 24:
                    prevalence[i] = 0.0031
                elif i <= 29:
                    prevalence[i] = 0.0052
                elif i <= 34:
                    prevalence[i] = 0.0092
                elif i <= 39:
                    prevalence[i] = 0.0201
                elif i <= 44:
                    prevalence[i] = 0.0497
                elif i <= 49:
                    prevalence[i] = 0.0607
                elif i <= 54:
                    prevalence[i] = 0.0839
                elif i <= 59:
                    prevalence[i] = 0.1160
                elif i <= 64:
                    prevalence[i] = 0.1552
                elif i <= 69:
                    prevalence[i] = 0.2005
                elif i <= 74:
                    prevalence[i] = 0.2561
                elif i <= 79:
                    prevalence[i] = 0.3128
                elif i <= 84:
                    prevalence[i] = 0.3655
                elif i <= 89:
                    prevalence[i] = 0.4115
                else:
                    prevalence[i] = 0.4356





    ######################################  Japan #############################
    if country=='japan':

        if comorbidity=='diabetes':
            for i in range(101):
                if i<20:
                    prevalence[i]= 0.02*(i/20.)
                elif i<30:
                    prevalence[i]=0.02
                elif i<40:
                    prevalence[i]=0.04
                elif i<50:
                    prevalence[i]=0.07
                elif i<60:
                    prevalence[i]=0.13
                elif i<70:
                    prevalence[i]=0.22
                else:
                    prevalence[i]=0.23

        elif comorbidity=='hypertension':
            for i in range(101):
                if i<20:
                    prevalence[i]= 0.085*(i/20.)
                elif i<35:
                    prevalence[i]=(1*0.15 + 1*0.02)/(1+1) # = 0.085
                elif i<50:
                    prevalence[i]=(1*0.3 + 1*0.1)/(1+1) # = 0.2
                elif i<65:
                    prevalence[i]=(1*0.67 + 1*0.53)/(1+1) # = 0.6
                else:
                    prevalence[i]=0.8

    ######################################  uk ################################
    if country=='uk':

        if comorbidity=='diabetes':
            for i in range(101):
                if i<16:
                    prevalence[i]= 0.003*(i/16.)
                elif i<25:
                    prevalence[i]=0.003
                elif i<44:
                    prevalence[i]=0.015
                elif i<65:
                    prevalence[i]=0.091
                else:
                    prevalence[i]=0.16

        elif comorbidity=='hypertension':
            for i in range(101):
                if i<16:
                    prevalence[i]= 0.02*(i/16.)
                elif i<25:
                    prevalence[i]=0.02
                elif i<44:
                    prevalence[i]=0.09
                elif i<65:
                    prevalence[i]=0.26
                else:
                    prevalence[i]=0.46

    ######################################  France ################################
    if country=='France':

        if comorbidity=='diabetes':
            for i in range(101):
                if i<40:
                    prevalence[i]= 0
                elif i<45:
                    prevalence[i]=0.01
                elif i<50:
                    prevalence[i]=0.025
                elif i<55:
                    prevalence[i]=0.05
                elif i<60:
                    prevalence[i]=0.075
                elif i<65:
                    prevalence[i]=0.12
                elif i<70:
                    prevalence[i]=0.14
                elif i<75:
                    prevalence[i]=0.16
                elif i<80:
                    prevalence[i]=0.18
                elif i<85:
                    prevalence[i]=0.17
                elif i<90:
                    prevalence[i]=0.14
                else:
                    prevalence[i]=0.09

        elif comorbidity=='hypertension':
            for i in range(101):
                if i<35:
                    prevalence[i]= 0.161*(i/35.)
                elif i<45:
                    prevalence[i]=0.161
                elif i<55:
                    prevalence[i]=0.39
                elif i<65:
                    prevalence[i]=0.56
                else:
                    prevalence[i]=0.75


    ######################################  China #############################
    if country=='China':
        #print ("LOL")
        dummy_value_to_work_with_china_function=0
        if comorbidity=='diabetes':
            for i in range(101):
                if i <= 4:
                    prevalence[i] = 0.0000
                elif i <= 9:
                    prevalence[i] = 0.0001
                elif i <= 14:
                    prevalence[i] = 0.0007
                elif i <= 19:
                    prevalence[i] = 0.0069
                elif i <= 24:
                    prevalence[i] = 0.0259
                elif i <= 29:
                    prevalence[i] = 0.0403
                elif i <= 34:
                    prevalence[i] = 0.0499
                elif i <= 39:
                    prevalence[i] = 0.0591
                elif i <= 44:
                    prevalence[i] = 0.0694
                elif i <= 49:
                    prevalence[i] = 0.0834
                elif i <= 54:
                    prevalence[i] = 0.0996
                elif i <= 59:
                    prevalence[i] = 0.1144
                elif i <= 64:
                    prevalence[i] = 0.1249
                elif i <= 69:
                    prevalence[i] = 0.1318
                elif i <= 74:
                    prevalence[i] = 0.1348
                elif i <= 79:
                    prevalence[i] = 0.1357
                elif i <= 84:
                    prevalence[i] = 0.1349
                elif i <= 89:
                    prevalence[i] = 0.1303
                else:
                    prevalence[i] = 0.1226
        elif comorbidity=='hypertension':
            prevalence= p_hypertension_china(dummy_value_to_work_with_china_function)
        elif comorbidity=='CVD':
            for i in range(101):
                if i <= 4:
                    prevalence[i] = 0.0010
                elif i <= 9:
                    prevalence[i] = 0.0039
                elif i <= 14:
                    prevalence[i] = 0.0075
                elif i <= 19:
                    prevalence[i] = 0.0100
                elif i <= 24:
                    prevalence[i] = 0.0108
                elif i <= 29:
                    prevalence[i] = 0.0135
                elif i <= 34:
                    prevalence[i] = 0.0168
                elif i <= 39:
                    prevalence[i] = 0.0232
                elif i <= 44:
                    prevalence[i] = 0.0448
                elif i <= 49:
                    prevalence[i] = 0.0655
                elif i <= 54:
                    prevalence[i] = 0.0946
                elif i <= 59:
                    prevalence[i] = 0.1336
                elif i <= 64:
                    prevalence[i] = 0.1816
                elif i <= 69:
                    prevalence[i] = 0.2370
                elif i <= 74:
                    prevalence[i] = 0.2953
                elif i <= 79:
                    prevalence[i] = 0.3528
                elif i <= 84:
                    prevalence[i] = 0.3999
                elif i <= 89:
                    prevalence[i] = 0.4250
                else:
                    prevalence[i] = 0.4325

    ######################################  Italy #############################
    if country=='Italy':
        
        if comorbidity=='diabetes':
            for i in range(101):
                if i <= 4:
                    prevalence[i] = 0.0001
                elif i <= 9:
                    prevalence[i] = 0.0009
                elif i <= 14:
                    prevalence[i] = 0.0024
                elif i <= 19:
                    prevalence[i] = 0.0091
                elif i <= 24:
                    prevalence[i] = 0.0264
                elif i <= 29:
                    prevalence[i] = 0.0356
                elif i <= 34:
                    prevalence[i] = 0.0392
                elif i <= 39:
                    prevalence[i] = 0.0428
                elif i <= 44:
                    prevalence[i] = 0.0489
                elif i <= 49:
                    prevalence[i] = 0.0638
                elif i <= 54:
                    prevalence[i] = 0.0893
                elif i <= 59:
                    prevalence[i] = 0.1277
                elif i <= 64:
                    prevalence[i] = 0.1783
                elif i <= 69:
                    prevalence[i] = 0.2106
                elif i <= 74:
                    prevalence[i] = 0.2407
                elif i <= 79:
                    prevalence[i] = 0.2851
                elif i <= 84:
                    prevalence[i] = 0.3348
                elif i <= 89:
                    prevalence[i] = 0.3517
                else:
                    prevalence[i] = 0.3354

        elif comorbidity=='hypertension':
            for i in range(101):
                if i<35:
                    prevalence[i]= 0.14*(i/35.)
                elif i<39:
                    prevalence[i]=0.14
                elif i<44:
                    prevalence[i]=0.1
                elif i<49:
                    prevalence[i]=0.16
                elif i<54:
                    prevalence[i]=0.3
                else:
                    prevalence[i]=0.34

        elif comorbidity=='CVD':
            for i in range(101):
                if i <= 4:
                    prevalence[i] = 0.0021
                elif i <= 9:
                    prevalence[i] = 0.0020
                elif i <= 14:
                    prevalence[i] = 0.0015
                elif i <= 19:
                    prevalence[i] = 0.0027
                elif i <= 24:
                    prevalence[i] = 0.0037
                elif i <= 29:
                    prevalence[i] = 0.0074
                elif i <= 34:
                    prevalence[i] = 0.0106
                elif i <= 39:
                    prevalence[i] = 0.0156
                elif i <= 44:
                    prevalence[i] = 0.0390
                elif i <= 49:
                    prevalence[i] = 0.0569
                elif i <= 54:
                    prevalence[i] = 0.0816
                elif i <= 59:
                    prevalence[i] = 0.1249
                elif i <= 64:
                    prevalence[i] = 0.1904
                elif i <= 69:
                    prevalence[i] = 0.2735
                elif i <= 74:
                    prevalence[i] = 0.3618
                elif i <= 79:
                    prevalence[i] = 0.4350
                elif i <= 84:
                    prevalence[i] = 0.4762
                elif i <= 89:
                    prevalence[i] = 0.4916
                else:
                    prevalence[i] = 0.4825

    ######################################  Maharashtra #############################
    if country=='Maharashtra':    
        print("LOL13")
        if comorbidity=='diabetes':
            
            prevalence=np.array([i*1.63/15 for i in range(15)]+[1.63,1.93,2.00,2.49,2.50,1.21,1.82,2.57,
                        2.21,2.42,3.22,2.37,3.52,3.00,4.18,5.12,
                        3.83,4.29,6.82,5.77,7.53,7.83,8.69,8.88,
                        8.03,8.14,8.50,9.97,10.05,14.43,15.29,
                        13.44,16.39,18.65,15.11,17.66,24.05,16.79,
                        18.53,21.46, 21.55, 22.54, 23.56, 24.59, 
                        25.64, 26.71, 27.79, 28.9,  30.02, 31.16, 
                        32.31,33.49, 34.68, 35.88, 37.1,  38.34,
                        39.6,  40.87, 42.15, 43.45, 44.77, 46.1,
                        47.44, 48.8,50.18, 51.56, 52.97, 54.38, 
                        55.81, 57.26, 58.72, 60.19, 61.67, 63.17,
                        64.68, 66.21,67.75, 69.3, 70.86, 72.43, 
                        74.02, 75.62, 77.23, 78.86, 80.5,82.14])
            for i in range(55,101):
                prevalence[i]=0.5*(prevalence[i]-prevalence[54]) + prevalence[54]
            prevalence=np.array(prevalence)
            prevalence=prevalence/100
            
        elif comorbidity=='hypertension':
            
            prevalence=[i*1.23/15 for i in range(15)]+[1.2300e+00,  1.6300e+00,  2.0500e+00,  2.5100e+00,
                        3.0100e+00,  3.5300e+00,  4.0900e+00,  4.6700e+00 , 5.3000e+00,  5.9500e+00,
                        6.6300e+00 , 7.3500e+00,  8.1000e+00,  8.8900e+00 , 9.7000e+00,  1.0550e+01,
                        1.1430e+01,  1.2340e+01,  1.3290e+01,  1.4260e+01 , 1.5270e+01,  1.6310e+01,
                        1.7390e+01,  1.8490e+01,  1.9630e+01,  2.0800e+01 , 2.2010e+01,  2.3240e+01,
                        2.4510e+01,  2.5810e+01,  2.7140e+01,  2.8510e+01 , 2.9900e+01,  3.1330e+01,
                        3.2800e+01,  3.4290e+01 , 3.5820e+01,  3.7380e+01 , 3.8970e+01,  4.0590e+01,
                        4.2250e+01,  4.3940e+01 , 4.5660e+01,  4.7410e+01 , 4.9190e+01,  5.1010e+01,
                        5.2860e+01,  5.4740e+01 , 5.6660e+01,  5.8610e+01 , 6.0590e+01,  6.2600e+01,
                        6.4640e+01,  6.6720e+01 , 6.8830e+01,  7.0970e+01 , 7.3140e+01,  7.5350e+01,
                        7.7590e+01,  7.9860e+01 , 8.2160e+01,  8.4500e+01 , 8.6860e+01,  8.9260e+01,
                        9.1700e+01,  9.4160e+01 , 9.6660e+01,  9.9190e+01 , 1.0175e+02,  1.0434e+02,
                        1.0697e+02,  1.0963e+02 , 1.1232e+02,  1.1504e+02 , 1.1780e+02,  1.2058e+02,
                        1.2341e+02,  1.2626e+02 , 1.2914e+02,  1.3206e+02 , 1.3501e+02,  1.3799e+02,
                        1.4101e+02,  1.4405e+02 , 1.4713e+02,  1.5024e+02]
            for i in range(55,101):
                prevalence[i]=0.2*(prevalence[i]-prevalence[54]) + prevalence[54]
            prevalence=np.array(prevalence)
        
            prevalence=prevalence/100
    ######################################  Uttar Pradesh ################################
    if country=='up':

        warning_string= " 1. Prevalence assumed to be linerly increasing between ages 0-18 and 54+"

        if comorbidity=='diabetes':
                for i in range(101):
                    if i<15:
                        prevalence[i]= 0.025*(i/15.)
                    elif i<19:
                        prevalence[i]=0.025
                    elif i<24:
                        prevalence[i]=0.0288
                    elif i<29:
                        prevalence[i]=0.0412
                    elif i<34:
                        prevalence[i]=0.0595
                    elif i<39:
                        prevalence[i]=0.0781
                    elif i<44:
                        prevalence[i]=0.111
                    elif i<49:
                        prevalence[i]=0.139
                    elif i<54:
                        prevalence[i]=0.206
                    else:
                        prevalence[i]=0.206+0.5*((i-54)/46)
                prevalence=np.array(prevalence)
                #prevalence=prevalence/100
                

        elif comorbidity=='hypertension':
                for i in range(101):
                    if i<15:
                        prevalence[i]= 0.0353*(i/15.)
                    elif i<19:
                        prevalence[i]=0.0353
                    elif i<24:
                        prevalence[i]=0.0493
                    elif i<29:
                        prevalence[i]=0.080
                    elif i<34:
                        prevalence[i]=0.121
                    elif i<39:
                        prevalence[i]=0.156
                    elif i<44:
                        prevalence[i]=0.202
                    elif i<49:
                        prevalence[i]=0.240
                    elif i<54:
                        prevalence[i]=0.248
                    else:
                        prevalence[i]=0.206+0.5*((i-54)/46)
                prevalence=np.array(prevalence)
                #prevalence=prevalence/100
    if warning:
        print ("Warning: \n", warning_string)


    #status = (np.random.rand(age.shape[0]) < prevalence[age]).astype(np.int)
    return prevalence


LOL


In [134]:
#from sample_comorbidities import sample_joint_comorbidities, p_comorbidity
a= p_comorbidity("Maharashtra", 'diabetes', warning=False)
country='up'
n_ages = 101
age_groups = tuple([np.where(age == i)[0] for i in range(0, n_ages)])
    #    age_group_sizes = np.array([age_groups[i].shape[0] for i in range(0, n_ages)])
    #    diabetes = sample_comorbidities.sample_diabetes_china(age)
    #    hypertension = sample_comorbidities.sample_hypertension_china(age)
diabetes, hypertension = sample_joint_comorbidities(age, country)

LOL13
LOL2


In [135]:
# Does number of ppl with diabetes make sense?
diabetes.sum()

921171

In [136]:
# Does number of ppl with hypertension make sense?
hypertension.sum()

1251575

In [137]:
import pickle
pickle.dump([age, households, diabetes, hypertension, age_groups ], open('up_population_10000000.pickle', "wb" ))

In [138]:
# Check that it is loading correctly from file
age, households, diabetes, hypertension, age_groups = pickle.load(open('{}_population_{}.pickle'.format(country, n), 'rb'))